An exacerbation's start can be spotted by the pattern [False, True] in Is Exacerbated. Similarly, it's [True, False] for an exacerbation's end.

In [1]:
import ex_labels_data
import numpy as np


In [2]:
df = ex_labels_data.load().reset_index()
df.head()


Exacerbated labels data from the predictive classifier has 14452 entries (1445 exacerbated, 13006 not exacerbated measurements, 1 NaN)


,ID,Date recorded,PatientNbr,Study,CalcDate,CalcDatedn,ScenType,Scenario,BaseExample,Measure,Frequency,Percentage,MSExample,Is Exacerbated
0,23,2015-08-05,1,SC,05-Aug-2015,1,0,Actual,0,NaN,0,0,0,False
1,23,2015-08-06,1,SC,06-Aug-2015,2,0,Actual,0,NaN,0,0,0,False
2,23,2015-08-07,1,SC,07-Aug-2015,3,0,Actual,0,NaN,0,0,0,False
3,23,2015-08-08,1,SC,08-Aug-2015,4,0,Actual,0,NaN,0,0,0,False
4,23,2015-08-09,1,SC,09-Aug-2015,5,0,Actual,0,NaN,0,0,0,False


In [10]:
def get_ex_state(row):
    if row["Is Exacerbated"] == True and row["Is Exacerbated Prev"] == False:
        return "start"
    elif row["Is Exacerbated"] == False and row["Is Exacerbated Prev"] == True:
        return "end"
    else:
        return "during"

In [8]:
test_df = df[df.ID == "23"].copy()
test_df["Is Exacerbated prev"] = test_df["Is Exacerbated"].shift(1)
test_df["Exacerbation state"] = test_df.apply(lambda x: get_ex_state(x), axis=1)


In [12]:
# Create a new column named Exacerbation state and fill it with nans
df["Exacerbation State"] = np.nan

# For each ID, fill the Exacerbation State column with the exacerbation state
# "start" if the index is in the ex_start_indices array
# "end" if the index is in the ex_end_indices array
# "during" if the index is in neither array

for ID in df.ID.unique():
    df_for_ID = df[df.ID == ID].copy().reset_index(drop=True)
    df_for_ID["Is Exacerbated Prev"] = df_for_ID["Is Exacerbated"].shift(1)
    df_for_ID["Exacerbation State"] = df_for_ID.apply(lambda x: get_ex_state(x), axis=1)

    df.loc[df.ID == ID, "Exacerbation State"] = df_for_ID["Exacerbation State"]

df.head()


,ID,Date recorded,PatientNbr,Study,CalcDate,CalcDatedn,ScenType,Scenario,BaseExample,Measure,Frequency,Percentage,MSExample,Is Exacerbated,Exacerbation State
0,23,2015-08-05,1,SC,05-Aug-2015,1,0,Actual,0,NaN,0,0,0,False,during
1,23,2015-08-06,1,SC,06-Aug-2015,2,0,Actual,0,NaN,0,0,0,False,during
2,23,2015-08-07,1,SC,07-Aug-2015,3,0,Actual,0,NaN,0,0,0,False,during
3,23,2015-08-08,1,SC,08-Aug-2015,4,0,Actual,0,NaN,0,0,0,False,during
4,23,2015-08-09,1,SC,09-Aug-2015,5,0,Actual,0,NaN,0,0,0,False,during
